In [1]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2

In [2]:
model_path = "roboflow_model/roboflow_bus_weights.onnx"
session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])

model_inputs = session.get_inputs()
model_outputs = session.get_outputs()
input_names = [inp.name for inp in model_inputs]
output_names = [out.name for out in model_outputs]

print("Model Inputs:", input_names)
print("Model Outputs:", output_names)

Model Inputs: ['images']
Model Outputs: ['output0']


In [3]:
image_path="test_bus.jpg"
img = Image.open(image_path)
img = np.array(img).astype(np.float32)/255

img = np.expand_dims(img, axis=0)

img = np.transpose(img, (0, 3, 1, 2))


outputs = session.run(output_names, {input_names[0]: img})

In [4]:
output_data = np.array(outputs[0])
output_data = np.squeeze(output_data)
output_data = output_data.T

boxes = output_data[:, :4]
scores = output_data[:, 4]

In [5]:
CONF_THRESHOLD = 0.8
valid_indices = np.where(scores > CONF_THRESHOLD)[0]

valid_boxes = boxes[valid_indices]
valid_scores = scores[valid_indices]

print(f"Valid Detections: {len(valid_boxes)}")

Valid Detections: 9


In [6]:
image = cv2.imread("test_bus.jpg")
image = cv2.resize(image, (640, 640))

for i in range(len(valid_boxes)):
    x, y, w, h = valid_boxes[i]

    x1 = int(x - w / 2)
    y1 = int(y - h / 2)
    x2 = int(x + w / 2)
    y2 = int(y + h / 2)

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    label = f"Confidence: {valid_scores[i]:.2f}"
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

image=cv2.resize(image, (640*2, 640*2))

cv2.imshow("Detections", image)
cv2.waitKey(0)
cv2.destroyAllWindows()